# Actividades a realizar
* Evaluar la regresión logística vainilla en los conjuntos de prueba y train, indicar cuales son las
conclusiones de su uso.

* Evualar el modelo lineal, e indicar cuales son los problemas encontrados.

* Implentar el método SAFE para transformar las variables y evaluar su desempeño.

* Usar el modelo de bosque aleatorio supervisor flexible.

* Transformar las variables como se indica en los dos ejemplos posteriores y luego evaluar la regresión logística basada en variables transformadas y justificar si se puede utilizar para predecir si se cancelará una reserva.

#Instalación de librerias y librerias

In [1]:
pip install safe-transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from SafeTransformer import SafeTransformer
from sklearn.metrics import accuracy_score

#Exploración del dataset

Contamos con dos conjuntos de datos los cuales pertenecen a las reservaciones de hoteles, por lo que a continuación veremos por encima cuantos registros tenemos, cuántos son nulos, los nombres de las columnas y el tipo de dato que tienen asignado.

In [3]:
H1df = pd.read_csv('H1.csv')
H2df = pd.read_csv('H2.csv')

H1df = pd.concat([H1df, H2df])

H1df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0.0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0.0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0.0,...,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0.0,...,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0.0,...,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03


Nuestro conjunto de datos cuenta con 119390 registros de los cuales 488 registros tienen un valor nulo en la columna **Country**.

Por lo que debemos hacer algo con esos registros, en este caso opté por borrarlos ya que no influye demasiado en la muestra.

In [4]:
H1df.info()
H1df.dropna(inplace=True)

print("\nRegistros nulos detectados:\n")
H1df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119390 entries, 0 to 79329
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   IsCanceled                   119390 non-null  int64  
 1   LeadTime                     119390 non-null  int64  
 2   ArrivalDateYear              119390 non-null  int64  
 3   ArrivalDateMonth             119390 non-null  object 
 4   ArrivalDateWeekNumber        119390 non-null  int64  
 5   ArrivalDateDayOfMonth        119390 non-null  int64  
 6   StaysInWeekendNights         119390 non-null  int64  
 7   StaysInWeekNights            119390 non-null  int64  
 8   Adults                       119390 non-null  int64  
 9   Children                     119386 non-null  float64
 10  Babies                       119390 non-null  int64  
 11  Meal                         119390 non-null  object 
 12  Country                      118902 non-null  object 
 13  

IsCanceled                     0
LeadTime                       0
ArrivalDateYear                0
ArrivalDateMonth               0
ArrivalDateWeekNumber          0
ArrivalDateDayOfMonth          0
StaysInWeekendNights           0
StaysInWeekNights              0
Adults                         0
Children                       0
Babies                         0
Meal                           0
Country                        0
MarketSegment                  0
DistributionChannel            0
IsRepeatedGuest                0
PreviousCancellations          0
PreviousBookingsNotCanceled    0
ReservedRoomType               0
AssignedRoomType               0
BookingChanges                 0
DepositType                    0
Agent                          0
Company                        0
DaysInWaitingList              0
CustomerType                   0
ADR                            0
RequiredCarParkingSpaces       0
TotalOfSpecialRequests         0
ReservationStatus              0
Reservatio

# Eliminar variables insignificantes

A mi consideración borré las siguientes variables que a mi parecer no presentan tanta relevancia para la creación de mis modelos y sus predicciones. Las variables eliminadas son las siguientes:
* ArrivalDateYear
* ArrivalDateWeekNumber
* Agent
* Company
* DistributionChannel
* ReservationStatusDate
* StaysInWeekendNights

In [5]:
H1df.drop(columns = ['ArrivalDateYear', 'Agent', 'Company','ReservationStatusDate', 'DistributionChannel', 'ArrivalDateWeekNumber', 'StaysInWeekendNights'], inplace = True)

# División de las varibles
Aquí lo único que hice fue dividir el conjunto de datos en las variables independientes y la dependiente. En este caso mi única variable dependiente es la columna **IsCanceled** que nos indica si la reservación se canceló o no, mientras que el resto de columnas serán mis variables independientes.

In [6]:
X = H1df.drop(columns = ['IsCanceled']) #Variables independientes
y = H1df.IsCanceled #Variable dependiente

Debido a que tenemos variables categóricas (son de tipo objeto) debemos transformarlas a un tipo numérico para que nuestros modelos puedan procesarlas sin problemas.

Para ello haremos uso de la librería **One Hot Encoder** que se encargará de hacer la transformación de las variables. También haremos uso de la librería **make_column_transformer** qué hará el proceso todavía más fácil pues no tendremos que hacer manualmente todas las transformaciones sino que esta librería lo realizará todo, ahora únicamente debemos pasarle a esta librería qué método utilizar y cuales son las columnas categóricas.

In [7]:
column_transformer = make_column_transformer(
    (OneHotEncoder(),['Meal', 'ReservationStatus', 'Country', 'ArrivalDateMonth', 'MarketSegment', 'DepositType', 'CustomerType', 'ReservedRoomType', 'AssignedRoomType']), remainder = 'passthrough'
    )

Por último debemos aplicar la transformación de variables a nuestro conjunto de variables independientes con el método **fit_transform**.

In [8]:
X = column_transformer.fit_transform(X).toarray()

Con esto hecho ya tenemos nuestras variables listas para realizar el modelado y las predicciones.

In [9]:
X

array([[  1.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [  1.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [  1.  ,   0.  ,   0.  , ...,  75.  ,   0.  ,   0.  ],
       ...,
       [  1.  ,   0.  ,   0.  , ..., 157.71,   0.  ,   4.  ],
       [  1.  ,   0.  ,   0.  , ..., 104.4 ,   0.  ,   0.  ],
       [  0.  ,   0.  ,   1.  , ..., 151.2 ,   0.  ,   2.  ]])

In [10]:
y[:3]

0    0
1    0
2    0
Name: IsCanceled, dtype: int64

# Division de las variables en conjunto de entrenamiento y prueba

Una vez tenemos nuestras variables toca dividirlas en dos conjuntos los cuales serán:
* Conjunto de entrenamiento: Nos ayudará a entrenar el modelo con un tamaño del 70%.
* Conjunto de pruebas: Nos ayudará a validar las predicciones de nuestro modelo con un tamaño del 30%. 


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

# Estandarización de las variables
Para no tener errores o métricas muy diferentes en nuestros datos lo que haremos es escalar todas las variables para tener más o menos la misma métrica.

In [12]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Regresión Logística

La Regresión Logística es un Algoritmo Supervisado y se utiliza para clasificación, con lo que podremos predecir si un cliente cancelará su reservación o no.

In [13]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

predict = logistic_model.predict(X_test)
logistic_model.score(X_test, y_test)

1.0

Como podemos ver nuestro modelo sorprendentemente tiene un acierto en las predicciones del 100%, cosa que nos debería hacer levantar una ceja ya que extrañamente un modelo llega a tan alto porcentaje.

Esto quizá es un síntoma de sobreentrenamiento.

# Regresión Lineal

El único error en el que puedo pensar es que en teoría la regresión lineal solo la debemos emplear cuando nuestra variable dependiente es de continua es decir es numérica, pero en nuestro caso la variable dependiente es categórica o binaria por lo que no tiene mucho sentido aplicar un modelo de regresión lineal para este problema.


In [14]:
linear_model = LinearRegression()

X = H1df.drop(columns = ['IsCanceled'])
y = H1df.IsCanceled

X = column_transformer.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

linear_model.fit(X_train, y_train)

predict = linear_model.predict(X_test)
linear_model.score(X_test, y_test)

0.9971261136457124

# Bosque aleatorio
El clasificador de bosque aleatorio crea un conjunto de árboles de decisión a partir de un subconjunto seleccionado aleatoriamente del conjunto de entrenamiento. Es básicamente un conjunto de árboles de decisión (DT) de un subconjunto seleccionado aleatoriamente del conjunto de entrenamiento y luego recoge los votos de diferentes árboles de decisión para decidir la predicción final.

In [15]:
randomforest_model = RandomForestClassifier()

X = H1df.drop(columns = ['IsCanceled'])
y = H1df.IsCanceled

X = column_transformer.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

randomforest_model.fit(X_train, y_train)

predict = randomforest_model.predict(X_test)
randomforest_model.score(X_test, y_test)

1.0

# Implementación del método SAFE y regresión logística
Según a como entiendo después de aplicar el modelo de bosque aleatorio debemos emplear sus predicciones en el método SAFE para posteriormente hacer nuevamente la regresión logística con los datos transformados.

In [ ]:
safe_transformer = SafeTransformer(model=randomforest_model, penalty=1)

safe_transformer = safe_transformer.fit(pd.DataFrame(X_train))

X_train_transformed = safe_transformer.transform(X_train)

X_test_transformed = safe_transformer.transform(X_test)

model_transformed = LogisticRegression()

model_transformed = model_transformed.fit(X_train_transformed, y_train)

surrogate_predictions = model_transformed.predict(X_test_transformed)